<a href="https://colab.research.google.com/github/pattangeumdduck/stack_study_record/blob/main/aice_professionalstudy_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#이미지 Dataset 만들기 - glob과 Dataset from_tensor_slices활용하기
- 콘크리트 이미지 데이터를 가지고 모델 학습에 사용할 데이터셋을 만들어 이미지 분류 실습
- 우선, 콘크리트 이미지 데이터(깨진 콘크리트와 안깨진 콘크리트 데이터)를 glob, from_tensor_slices, Pipeline을 활용해서 Dataset만들기
- 그리고, 모델 학습하기 위해서는 Dataset 뿐 아니라 라벨정보도 필요하므로 라벨정보도 포함되도록 dataset만들고 이를 이용해 모델을 구축해보기

#학습목차
A. 이미지 데이터셋 만들기(라벨 정보 없음)
1. 필요한 라이브러리 임포트
2. 이미지 파일 가져오기 :concrete_image.zip
3. 이미지 파일 하나 읽어 이미지보기
4. glob과 from_tensor_slices, Pipeline 이용하여 이미지 데이터셋 만들기

B. 라벨정보 포함한 이미지 데이터셋 만들기(많은 수작업 필요)
1. glob이용하여 이미지 패스 읽기
2. shuffle
3. Train/ Test용 나누기
4. 이미지 라벨링 만들기
5. from_tensor_slices >map > cache > batch > shuffle > prefetch 이미지/라벨링 데이터셋 만들기

C. Build Model
1. Build Model
2. Callback
3. 모델학습
4. 성능 그래프
5. predict



In [ ]:
# 이미지 데이터셋 만들기(라벨 정보 없이)
import os
from glob import glob

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
#이미지 파일 가져오기 :concrete_image.zip
glob('concrete_image.zip')

[]

In [ ]:
#이미지 폴더 생성 및 concrete_image.zip 파일 압축 풀기 > 3000개 이미지로 실습을 위해 데이터 축소.

if not os.path.exists('IMAGE'):
  os.mkdir('IMAGE')
  !unzip concrete_imagez.zip -d IMAGE

unzip:  cannot find or open concrete_imagez.zip, concrete_imagez.zip.zip or concrete_imagez.zip.ZIP.


In [ ]:
#concrete_image Negative 폴더 안의 이미지 갯수
!ls -1 ./IMAGE/Negative/ | grep jpg | wc -1

#concrete_image Positive폴더 안의 이미지 갯수
!ls -1 ./IMAGE/Positive/ | grep jpg | wc -1

ls: cannot access './IMAGE/Negative/': No such file or directory
wc: invalid option -- '1'
Try 'wc --help' for more information.
wc: invalid option -- '1'
Try 'wc --help' for more information.
ls: cannot access './IMAGE/Positive/': No such file or directory


In [ ]:
#이미지 파일 하나 읽어 이미지 보기
#이미지 패스 지정
path = './IMAGE/Negative/00001.jpg'

In [ ]:
#이미지 읽어오기
gfile = tf.io.read_file(path)
iage = tf.io.decode_image(gfile)

NotFoundError: {{function_node __wrapped__ReadFile_device_/job:localhost/replica:0/task:0/device:CPU:0}} ./IMAGE/Negative/00001.jpg; No such file or directory [Op:ReadFile]

In [ ]:
 #이미지 shape 확ㅇㄴ
 image.shape
 #읽어온 이미지 보기
 plt.imshow(image)
plt.show()

In [ ]:
#glob과 from_tensor_slices, Pipeline 이용하여 이미지 데이터셋 만들기
#glob 활용하여 이미지 패스를 만든다/
#glob결과로 리스트를 리턴함
image_paths = glob('./IMAGE/*/*.jpg')

print(len(image_paths))
print(image_paths[-10:])

In [ ]:
#이미지 패스를 주면 이미지 읽고 반환하는 함수
def read_image(path):
  gfile = tf.io.read_file(path)
  image = tf.io.decode_image(gfile)
  return image

In [ ]:
#병렬화
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
#이미지 패스리스트를 from tensor slices 사용해 데이터셋 만들고
#map 함수를 사용하여 각 이미지 패스의 이미지들을 병렬로 읽어오기
dataset = tf.data.Datasets.from_tensor_slices(images_paths)
dataset = dataset.map(read_image, num_paralel_callse=AUTOTUNE)

In [ ]:
#데이터셋에서 1개 이미지 가져오기
tf_image = next(iter(dataset))
tf_image.shape

In [ ]:
#Negative 데이터 샘플
plt.imshow(tf_image)
plt.show()


In [ ]:
#이미지 패스의 이미지 읽고(map) 4개 배치 묶기
dataset = tf.data.Dataset.from_tensor_slices(image_paths)
dataset = dataset.map(read_image)
dataset = dataset.batch(4)

In [ ]:
#Negative 데이터 샘플들
for i in range(4):
  plt.imshow(tf_images[i])
  plt.show()

In [ ]:
#from_tensor_slices > map > cache > batch > shuffle > prefetch 형태로 사용
dataset = tf.data.Dataset.from_tensor_slices(image_paths) #입력 :이미지 패스 리스트
dataset = dataset.map(read_image, num_parallel_calls=AUTOTUNE) #이미지 패스의 각 이미지 읽기
dataset = dataset.cache()
dataset = dataset.batch(4)
dataset = dataset.shuffle(buffer_size=512)
dataset = dataset.prefetch(buffer_size = AUTOTUNE)

In [ ]:
#shuffle 시간 좀 걸림
tf_image = next(iter(dataset))
tf_images.shape

In [ ]:
#shuffle 이전에 첫 번째 이미지와 지금은 다른 이미지로 shuffle 된것 확인
plt.imshow(tf_images[0])
plt.show()

#B.라벨정보 포함한 이미지 데이터셋 만들기(수작업 필요)
- 이미지 데이터만 만들었으나, 지도학습을 위해선 이미지 라벨링이 필요함

- Data Preprocess : 아래 작업을 손수 작성해야 함
1. glob이용하여 이미지 패스 읽기
2. shuffle
3. Train/Test 비율로 나누기
4. 이미지 라벨링 만들기
5. from_tensor_slices >map > cache > batch > prefetch 파이프라인 사용하여 이미지/라벨링 데이터셋 만들기


In [ ]:
# Hyperparameter Tunning
num_epochs =10
batch_size = 32

learning_rate = 0.001
dropout = 0.5
input_shape = (227, 227, 3) #사이즈 확인
num_classes = 2 #positive, Negative

In [ ]:
#glob를 통해 이미지 패스 읽어오기
image_paths_list = glob('./IMAGE/*/*.jpg')
print(len(image_paths_list))

image_paths = np.random.permutataion(image_paths_list)
image_paths[:10]

In [ ]:
# 8 :2 비율로 Train, Test 이미지셋 나누기
TRAIN_SIZE = int(len(image_paths) * 0.8) #전체 row 수의 0.8배만큼을 Trian 나머지를 test
print(TRAIN_SIZE)
train_paths = image_paths[:TRAIN_SIZE]
test_paths = image_paths[TRAIN_SIZE:]

In [ ]:
len(train_paths), len(test_paths)

In [ ]:
#Positive, Negative 폴더 이름 반환하는 함수
#EX) ./IMAGE?Negative07269.jog --> /Negative 가져오는 함수 만들기
def get_class_name(path):
  name = os.path.dirname(path).split('/')[-1] #폴더명
  return name

In [ ]:
# get_class_name 함수 정상 동작 여부 확인
for path in train_paths[:4]:
  print(path, get_class_name(path))

In [ ]:
#class 이름 만들기
train_labes = [get_class_name{path} for path in train_paths]
class_names = np.unique(train_labes)
class_names

In [ ]:
# 원핫 인코딩 간단 변환 예제 (한번 알아보기)
print('Negative' == np.array(['Negative', 'Positve']))
print(('Negative' == np.array(['Negative', 'Positve'])).astype(int))

In [ ]:
#파일  패스에서 'Negative', 'Postive' 폴더부분을 읽고
# class_name 과 비교해서 (numpy broadcasting) onehot 만들어 리턴

def get_label(path):
  label_name = tf.strings.split(path, '/')[-2]
  onehot = tf.cast(label_name == calss_names , tf.unit8) #원 핫 인코딩
  # return tf.argmax(onehot) , 이번에는 원핫이 아닌 라벨 번호로 함
  return onehot #이번에는 onehot 으로

In [ ]:
def load_image_label(path):
  gfile = tf.io.read_file(path)
  image = tf.io.decode_image(gfile)
  image = tf.cast(image, tf.float32) / 255. #rescale

  label = get_label(path)
  return image, label

In [ ]:
#이미지 변환 처리: 여기에서는 사용하지 않음
def image_preprocess(image, label):
  image = tf.image.random_flip_up_down(image)
  image = tf.image.random_flip_left_right(image)
  return image, label

In [ ]:
#load image label함수 잘 동작하는지 확인하기
load_image_label('./IMAGE/Negative/00472.jpg')

In [ ]:
#병렬화
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
# from_tensor_slices > map > cache > batch > shuffle > prefetch 형태로 사용

train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls = AUTOTUNE)
#train_dataset = train_dataset.map(image_preprocess, num_parallel_calls = AUTOTUNE)
train_dataset = traIn_dataset.cache()
train_dataset = train_dataset.shuffle(buffer_size = 512)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(buffer_size = AUTOTUNE)
#

In [ ]:
#from_tensor_slices > map > cache > batch >prefetch 형태로 사용
#test set은 shuffle 하지 않음

test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls = AUTOTUNE)
test_dataset = test_dataset.cache()
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.prefetch(buffer_size = AUTOTUNE)

In [ ]:
#샘플 이미지 확인
i = 0
for batch_img, batch_label in train_dataset.stake(1):
  if i == 0:
    print(batch_img[i].shape)
    plt.imshow(batch_img[i])
  i = i + 1

In [ ]:
#Build 모델
#우선 Functional API 모델 정의
inputs = tf.keras.layers.Input(input_shape)

net = tf.keras.layers.Conv2D(32, (3,3), padding = 'SAME')(inputs) #227 * 227 * 32
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.Conv2D(32,(3,3), padding = 'SAME')(net) #227 * 227 * 32
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.MaxPooling2D(pool_size = (2 , 2))(net) #113 * 113 * 32
net = tf.keras.layers.Dropout(dropout_rate)(net)

net = tf.keras.layers.Conv2D(64, (3,3), padding = 'SAME')(net) #113 * 113 * 64
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.Conv2D(64, (3,3), padding = 'SAME')(net) #113 * 113 * 64
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.MaxPooling2D(pool_size = (2 , 2))(net) #56 *56 * 64
net = tf.keras.layers.Dropout(dropout_rate)(net)

net = tf.keras.layers.Flatten()(net)
net = tf.keras.layers.Dense(512)(net)
net = tf.keras.layers.Activation('relu')(net)
net = tf.keras.layers.Dropout(dropout_rate)(net)
net = tf.keras.layers.Dense(num_classes)(net)
net = tf.keras.layers.Activation('softmax')(net)

model = tf.keras.Model(inputs = inputs, outputs = net, name = 'Basic CNN')

NameError: name 'tf' is not defined

In [ ]:
#모델 컴파일
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
#Callback : EarlyStopping , ModelCheckpoint, ReduceLROnPlateau

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

#Early Stopping
es = EarlyStopping(monitor='val_loss', mode ='min', verbose =1, patience =3)

#ModelCheckPoint
checkpoint_path = "my_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath = checkpoint_path,
                             save_weights_only = True,
                             save_best_only = True,
                             monitor = 'val_loss',
                             verbose =1)


#ReduceLROnPlateau :val_loss 가 2번 이상 감소되지 않으면 lr * factor = lr 새로운 lr로 변경해서 학습 진행
lrReducer = ReduceLROnPlateau(monitor='val_loss', factor = 0.5, patience =2, min_lr =0.0001)


ValueError: When using `save_weights_only=True` in `ModelCheckpoint`, the filepath provided must end in `.weights.h5` (Keras weights format). Received: filepath=my_checkpoint.ckpt

In [ ]:
#모델 학습
#에포크 10 배치사이즈 32 데이터 학습시간이 오래 걸려 take(10)활용함 ...정확도 50도 안됨 ㅜㅜ
#나중에 take(10) 뺴도록해
history = model.fit(
    train_dataset.take(10),
    validation_data = (test_dataset.take(10)),
    epochs =5,
    batch_size =batch_size,
    callbacks = [es,chechpoint, lrReducer]
    )

NameError: name 'model' is not defined

In [4]:
#성능 그래프
history.history.keys()
plt.plot(history.history['accuracy']m label ="Accuracy")
plt.plot(history.history['val_accuracy'], label= 'Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accurcay')
plt.show()

NameError: name 'history' is not defined

In [ ]:
#너무 적은 데이터로 학습하니 성능이 50% 정도 나옴
plt.figure(figsize=(16,30))
for batch_img, batch_label in test_dataset.take(1):
  for i in ragne(len(batch_img)):
    pred = model.predict(batch_img[i].numpy().reshape(-1, 227, 227 , 3))
    pred_t = np.argmax(pred)
    plt.subplot(10, 5, i+1)
    plt.title(f'True Value:{np.argmax(batch_label[i])}, Pred Value: {pred_t}')
    plt.imshow(batch_img[i])
    plt.axis('off')

# 배운 내용 정리
1. glob과 Dataset from_tensor_slices 활용해서 이미지 Dataset 만들어 봄
2. glob, from_tensor_slices 함수를 이용하여 필요한 데이터를 Datasetdksdmfh dlfrdjdhrl
3. 이미지 파일명 읽어와 화면에 보여주기 : tf.io.read_file, tf.io.decode_image, plt.imshow
4. Tensorflow Dataset활용해서 읽어온 데이터를 파이프라인을 통하여 shuffle, batch, cache, prefetch된 Dataset을 만들 수 있다.
5. 모델 학습을 위해서는 라벨 정보도 필요하므로
6. 라벨 정보 포함해서 glob, from_tensor_slices, Pipeline(map, cache, prefetch된 Dataset을 만들 수 있다.)
7. 하지만 glob, from_tensor_slices, Pipeline, 라벨 코딩등 굉장히 번거롭고 수작업이 필요했으며
8. 이런 수작업을 쉽게 해줄 고수준의 API가 있음...